In [473]:
!pip install tweepy --quiet

In [474]:
import sys
sys.executable

'c:\\Users\\14708\\anaconda3\\python.exe'

In [475]:
import numpy as np 
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px 
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

pd.set_option('display.max_columns', 25)

import tweepy
from googletrans import Translator
from langdetect import detect
from translate import Translator

In [476]:
main_df=pd.read_csv(r"C:\Users\14708\Downloads\Tata and Hyundai cars - data.xlsx - Data.csv")
df = main_df.copy()

## Data Exploration, Preparation

In [477]:
df.head()

,Date,Headline,URL,Opening Text,Hit Sentence,Source,Influencer,Country,Subregion,Language,Reach,Desktop Reach,Mobile Reach,Twitter Social Echo,Facebook Social Echo,Reddit Social Echo,National Viewership,Engagement,Sentiment,Keywords,State
0,28-Sep-2023 02:08PM,Kia और Hyundai ने वापस बुलाईं अपनी 35 लाख कारे...,https://hindi.pardaphash.com/kia-and-hyundai-r...,Kia and Hyundai Recalled Cars: वाहन निर्माता क...,Kia and Hyundai Recalled Cars: वाहन निर्माता क...,Pardaphash,NaN,India,Delhi,Hindi,31307.0,1279.0,30028.0,0.0,0.0,0.0,0,NaN,Neutral,"Hyundai,Cars",Delhi
1,28-Sep-2023 01:46PM,"Tata Punch EV spied, may compete with MG Comen...",https://economictimes.indiatimes.com/industry/...,Tata Motors is reportedly developing an electr...,... Punch EV is expected to compete with the M...,The Economic Times,NaN,India,National,English,15403059.0,6171732.0,9231327.0,0.0,0.0,0.0,0,NaN,Neutral,"Tata,cars",Maharashtra
2,28-Sep-2023 01:30PM,Tata Avinya Reviews On Road Price 2023 Mileage...,https://badisoch.in/automobile-news/tata-aviny...,... (adsbygoogle = window.adsbygoogle || []).p...,"... , despite most carmakers focusing on large...",Badi Soch,NaN,India,NaN,English,1128280.0,171549.0,956731.0,0.0,0.0,0.0,0,NaN,Neutral,"cars,Tata",NaN
3,28-Sep-2023 01:24PM,NaN,http://twitter.com/automobilindia8/statuses/17...,NaN,Engine Fire-Related Risks Force Kia & Hyundai ...,Twitter,@automobilindia8,India,NaN,English,2.0,NaN,NaN,NaN,NaN,NaN,0,0.0,Neutral,"Hyundai,Cars,hyundai",Karnataka
4,28-Sep-2023 01:22PM,NaN,http://twitter.com/MalakpetD/statuses/17073021...,NaN,"@srinualavilli @BMTC_BENGALURU Mahindra, Tata,...",Twitter,@MalakpetD,India,NaN,English,506.0,NaN,NaN,NaN,NaN,NaN,0,0.0,Neutral,"Tata,cars",Telangana


In [478]:
df.tail()

,Date,Headline,URL,Opening Text,Hit Sentence,Source,Influencer,Country,Subregion,Language,Reach,Desktop Reach,Mobile Reach,Twitter Social Echo,Facebook Social Echo,Reddit Social Echo,National Viewership,Engagement,Sentiment,Keywords,State
1523,12-Sep-2023 06:20AM,MG3 hatchback takes out ‘cheapest car to run’ ...,https://bulletinreporter.com/mg3-hatchback-tak...,A small car has claimed the title of the cheap...,"... Costs Survey, which lists the cost of runn...",Bulletin Reporter,Kemal Pasha,India,NaN,English,2074.0,2074.0,0.0,0.0,0.0,0.0,0,NaN,Positive,"Hyundai,cars",NaN
1524,12-Sep-2023 05:24AM,MG3 hatchback named by RACV as the most cost-e...,https://vervetimes.com/mg3-hatchback-named-by-...,A small car has claimed the title of the cheap...,"... Costs Survey, which lists the cost of runn...",Verve times - Latest News around the World,NaN,India,NaN,English,26425.0,7491.0,18934.0,0.0,0.0,0.0,0,NaN,Positive,"Hyundai,cars",NaN
1525,12-Sep-2023 04:37AM,RACV names MG3 hatchback as the cheapest vehic...,https://newsconcerns.com/racv-names-mg3-hatchb...,A small car has claimed the title of the cheap...,"... Costs Survey, which lists the cost of runn...",News Concerns - Latest News Update,Charlie Brown,India,Madhya Pradesh,English,9323.0,2527.0,6796.0,0.0,0.0,0.0,0,NaN,Positive,"Hyundai,cars",Madhya Pradesh
1526,12-Sep-2023 04:04AM,Tata Power EZ App Electrifies Green Revolution...,https://thenfapost.com/tata-power-ez-app-elect...,Tata Power EZ App Electrifies Green Revolution...,Charging hubs for leading cab fleet operators....,NFAPost,N V Vijayakumar,India,Bangalore,English,851.0,851.0,0.0,0.0,0.0,0.0,0,NaN,Neutral,"Tata,cars",Karnataka
1527,12-Sep-2023 02:44AM,NaN,http://twitter.com/poojashah0998/statuses/1701...,NaN,RT @ShamsNaqvi: One of the most awaited cars t...,Twitter,@poojashah0998,India,NaN,English,45.0,NaN,NaN,NaN,NaN,NaN,0,NaN,Positive,"cars,Tata",Maharashtra


In [479]:
df.sample(5)

,Date,Headline,URL,Opening Text,Hit Sentence,Source,Influencer,Country,Subregion,Language,Reach,Desktop Reach,Mobile Reach,Twitter Social Echo,Facebook Social Echo,Reddit Social Echo,National Viewership,Engagement,Sentiment,Keywords,State
767,15-Sep-2023 08:05PM,"Meerut gang of Delhi NCR car lifters busted, 8...",https://theprint.in/india/meerut-gang-of-delhi...,"Noida, Sep 15 (PTI) With the arrest of eight s...",... Delhi National Capital Region (NCR). The p...,ThePrint,NaN,India,NaN,English,4519030.0,556217.0,3962813.0,0.0,0.0,0.0,0,NaN,Negative,"cars,Hyundai",Delhi
42,27-Sep-2023 07:34PM,NaN,https://twitter.com/rrajeshroy/statuses/170703...,NaN,QT @ActualIndia: Just a normal accident #Swift...,Twitter,@rrajeshroy,India,NaN,English,32.0,NaN,NaN,NaN,NaN,NaN,0,NaN,Negative,"cars,Tata",NaN
550,19-Sep-2023 09:58AM,NaN,http://twitter.com/thatdilligirl/statuses/1703...,NaN,@Iamsamirarora Try Tata cars for automobiles a...,Twitter,@thatdilligirl,India,NaN,English,49.0,NaN,NaN,NaN,NaN,NaN,0,0.0,Positive,"Tata,cars",Delhi
207,25-Sep-2023 10:48AM,NaN,http://twitter.com/jayeshkhunt20/statuses/1706...,NaN,RT @RajeshK_Chaubey: Still @TataMotors and @Ta...,Twitter,@jayeshkhunt20,India,NaN,English,291.0,NaN,NaN,NaN,NaN,NaN,0,NaN,Negative,"tata,cars",Gujarat
167,25-Sep-2023 11:37PM,NaN,http://twitter.com/jayeshkhunt20/statuses/1706...,NaN,RT @DAdavaram: @RNTata2000 RESPECT SIR..I BROU...,Twitter,@jayeshkhunt20,India,NaN,English,291.0,NaN,NaN,NaN,NaN,NaN,0,NaN,Neutral,"TATA,CARS",Gujarat


In [480]:
df.iloc[0]

Date                                                  28-Sep-2023 02:08PM
Headline                Kia और Hyundai ने वापस बुलाईं अपनी 35 लाख कारे...
URL                     https://hindi.pardaphash.com/kia-and-hyundai-r...
Opening Text            Kia and Hyundai Recalled Cars: वाहन निर्माता क...
Hit Sentence            Kia and Hyundai Recalled Cars: वाहन निर्माता क...
Source                                                         Pardaphash
Influencer                                                            NaN
Country                                                             India
Subregion                                                           Delhi
Language                                                            Hindi
Reach                                                             31307.0
Desktop Reach                                                      1279.0
Mobile Reach                                                      30028.0
Twitter Social Echo                   

In [481]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528 entries, 0 to 1527
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  1528 non-null   object 
 1   Headline              710 non-null    object 
 2   URL                   1528 non-null   object 
 3   Opening Text          710 non-null    object 
 4   Hit Sentence          1528 non-null   object 
 5   Source                1528 non-null   object 
 6   Influencer            1065 non-null   object 
 7   Country               1528 non-null   object 
 8   Subregion             321 non-null    object 
 9   Language              1528 non-null   object 
 10  Reach                 1519 non-null   float64
 11  Desktop Reach         710 non-null    float64
 12  Mobile Reach          710 non-null    float64
 13  Twitter Social Echo   710 non-null    float64
 14  Facebook Social Echo  710 non-null    float64
 15  Reddit Social Echo   

In [482]:
# Checking for Null values
df.isna().sum()

Date                       0
Headline                 818
URL                        0
Opening Text             818
Hit Sentence               0
Source                     0
Influencer               463
Country                    0
Subregion               1207
Language                   0
Reach                      9
Desktop Reach            818
Mobile Reach             818
Twitter Social Echo      818
Facebook Social Echo     818
Reddit Social Echo       818
National Viewership        0
Engagement              1176
Sentiment                  0
Keywords                   0
State                    500
dtype: int64

In [483]:
df = df[['Date', 'Headline', 'URL', 'Opening Text', 'Hit Sentence', 'Source',
       'Influencer', 'Country', 'Subregion', 'State', 'Language', 'Sentiment', 'Keywords',
       'Reach', 'Desktop Reach', 'Mobile Reach', 'Twitter Social Echo',
       'Facebook Social Echo', 'Reddit Social Echo', 'National Viewership',
       'Engagement'
       ]]

In [484]:
# Deleting the column Country
df.drop(columns='Country', inplace=True)

In [485]:
# Deleting headline column & opening text columns
df.drop(columns=['Headline', 'Opening Text'], inplace=True)

In [486]:
print('State Missing:',df['State'].isna().sum())
print('Subregion Missing:',df['Subregion'].isna().sum())

# These two are similar columns wioth large amounts of null values, thus one good way is to fill the null values with the values in the relevant column.
df['State'].fillna(df['Subregion'], inplace=True)
df['Subregion'].fillna(df['State'], inplace=True)

print(df['State'].isna().sum())
print(df['Subregion'].isna().sum())

# makes no sense to keep the subregion column 
df.drop(columns='Subregion', inplace=True)

State Missing: 500
Subregion Missing: 1207
497
497


In [487]:
# Check for duplicates
df.duplicated().sum()

0

In [488]:
# categorical columns
cat_cols = df.select_dtypes('object').columns
print(cat_cols)

# numerical columns
numeric_cols = df.select_dtypes(include=np.number).columns
print(numeric_cols)

Index(['Date', 'URL', 'Hit Sentence', 'Source', 'Influencer', 'State',
       'Language', 'Sentiment', 'Keywords'],
      dtype='object')
Index(['Reach', 'Desktop Reach', 'Mobile Reach', 'Twitter Social Echo',
       'Facebook Social Echo', 'Reddit Social Echo', 'National Viewership',
       'Engagement'],
      dtype='object')


In [489]:
df['Source'].value_counts()

Twitter                818
India Posts English     89
The Times of India      32
Cartoq                  21
Todays Chronic          14
                      ... 
TechCodex                1
Prameyanews              1
Orissa Diary             1
ODISHABARTA.COM          1
NFAPost                  1
Name: Source, Length: 230, dtype: int64

In [490]:
df.isna().sum()

Date                       0
URL                        0
Hit Sentence               0
Source                     0
Influencer               463
State                    497
Language                   0
Sentiment                  0
Keywords                   0
Reach                      9
Desktop Reach            818
Mobile Reach             818
Twitter Social Echo      818
Facebook Social Echo     818
Reddit Social Echo       818
National Viewership        0
Engagement              1176
dtype: int64

In [491]:
df[df['Source']=='Twitter'].isna().sum()

Date                      0
URL                       0
Hit Sentence              0
Source                    0
Influencer                0
State                   129
Language                  0
Sentiment                 0
Keywords                  0
Reach                     9
Desktop Reach           818
Mobile Reach            818
Twitter Social Echo     818
Facebook Social Echo    818
Reddit Social Echo      818
National Viewership       0
Engagement              504
dtype: int64

-----------------------------------------------------------------------------------------------------------------------------------------------------------

In [492]:
print(len(df['Keywords'].unique()))

print(len(df['Keywords'].str.lower().unique()))
df['Keywords']=df['Keywords'].str.lower()
df['Keywords'].str.lower().unique()

key_map = {
    'hyundai,cars':'hyundai', 'hyundai,cars,hyundai':'hyundai', 'cars,hyundai':'hyundai', 'hyundai,cars,cars':'hyundai',
    'tata,cars':'tata', 'cars,tata':'tata', 'cars,tata,hyundai':'hyundai_tata', 'cars,hyundai,tata':'hyundai_tata', 'tata,cars,tata':'tata',
    'cars,cars,tata':'tata', 'cars,hyundai,cars':'hyundai', 'cars,hyundai,cars,tata':'hyundai_tata', 'tata,cars,hyundai':'hyundai_tata',
    'tata,cars,cars':'tata', 'cars,tata,tata,hyundai':'hyundai_tata', 'hyundai,cars,tata,cars':'hyundai_tata', 'tata,tata,cars':'tata',
    'tata,cars,tata,cars':'tata','hyundai,cars,tata':'hyundai_tata', 'hyundai,tata,cars':'hyundai_tata', 'cars,cars,tata,tata':'tata',
    'cars,tata,tata':'tata', 'cars,tata,hyundai,cars':'hyundai_tata', 'cars,tata,cars':'tata','tata,cars,hyundai,cars':'hyundai_tata',
    'tata,hyundai,cars':'hyundai_tata', 'cars,hyundai,hyundai,cars':'hyundai', 'cars,tata,cars,hyundai':'hyundai_tata'
}

df['Keywords']=df['Keywords'].map(key_map)
df['Keywords'].unique()

65
28


array(['hyundai', 'tata', 'hyundai_tata'], dtype=object)

-----------------------------------------------------------------------------------------------------------------------------------------------------------

In [493]:
# Converting Hit Scentence to english for potential deep analysis in the future and to prevent data loss
df['Language'].value_counts()

English    1507
Hindi        14
Bengali       6
Marathi       1
Name: Language, dtype: int64

In [494]:
# Define a function to check if Hit Scentence column elements are not in English
def is_not_english(text):
    try:
        detected_language = detect(text)
        return detected_language != "en"
    except:
        return True  # If language detection fails, assume it's not English

# Apply the function to the 'Hit Sentence' column and create a boolean mask
non_english_mask = df['Hit Sentence'].apply(is_not_english)

# Filter the DataFrame to get rows with non-English text
non_english_texts = df[non_english_mask]

# Display the non-English texts
print(len(non_english_texts), ', ','This seems a small number, so lets translate these 26 elements into english')

27 ,  This seems a small number, so lets translate these 26 elements into english


In [495]:
def translate_to_english(text):
    translator = Translator(to_lang="en")
    translation = translator.translate(text)
    return translation

translations = []

for text in non_english_texts['Hit Sentence']:
    translation = translate_to_english(text)
    translations.append(translation)

non_english_texts['Translated Text'] = translations

non_english_texts['Translated Text']

C:\Users\14708\AppData\Local\Temp\ipykernel_22564\469313934.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_english_texts['Translated Text'] = translations


0       Kia and Hyundai Recalled Cars: वाहन निर्माता क...
123     "Hyundai Exter: नया एसयूवी आईएसयूवी मॉडल को ले...
127     Top 5 Affordable Electric Cars In India हमारी ...
199     ... Hyundai Cars Discount: हुंडई मोटर इंडिया ल...
226     ... में सीधे टाटा पंच ईवी के साथ होगा. टाटा पं...
227     RT @gaadiwaadi: 3 All-New Hyundai Cars Likely ...
241     RT @gaadiwaadi: 3 All-New Hyundai Cars Likely ...
343     Discounts on Hyundai Cars © महाराष्ट्र टाइम्स ...
445     हुंडई डिस्काउंट ऑफर्स ( Image Source : ABP Liv...
525     ... গাড়িরও দাম অনেকটাই বাড়বে বলে আশা করা হয়ে...
530     ... র্ভ কনসেপ্ট দ্বারা অনুপ্রাণিত হবে এবং একটি...
694     @Indian_Index 🚘(Aug23) Highest selling cars in...
695     🚘(Aug23) Highest selling cars in india: ➡️Maru...
728     ... SX 250, KTM Adventure 250, Bajaj Pulsar NS...
741     @Hitendra5325 @Akshara79485251 @tatamotorsindi...
954     ... Fearless AMT- 12,99,990 रुपये यह भी पढ़ें :...
1000    Diesel Cars Tax : 'যতদিন চাহিদা ততদিন ডিজেল গা...
1065    RT @BT

-----------------------------------------------------------------------------------------------------------------------------------------------------------

# Sentiment Analysis

In [516]:
df.groupby(['Keywords', 'Sentiment'], as_index=False).count()

,Keywords,Sentiment,Date,URL,Hit Sentence,Source,Influencer,State,Language,Reach,Desktop Reach,Mobile Reach,Twitter Social Echo,Facebook Social Echo,Reddit Social Echo,National Viewership,Engagement
0,hyundai,Negative,124,124,124,124,85,71,124,121,57,57,57,57,57,124,46
1,hyundai,Neutral,191,191,191,191,119,118,191,190,116,116,116,116,116,191,50
2,hyundai,Positive,162,162,162,162,72,85,162,162,133,133,133,133,133,162,29
3,hyundai_tata,Negative,6,6,6,6,5,4,6,5,2,2,2,2,2,6,2
4,hyundai_tata,Neutral,30,30,30,30,15,13,30,30,21,21,21,21,21,30,8
5,hyundai_tata,Positive,31,31,31,31,14,18,31,31,22,22,22,22,22,31,6
6,tata,Negative,128,128,128,128,124,105,128,127,8,8,8,8,8,128,66
7,tata,Neutral,629,629,629,629,501,482,629,626,192,192,192,192,192,629,89
8,tata,Positive,227,227,227,227,130,135,227,227,159,159,159,159,159,227,56
